In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import exp

np.set_printoptions(suppress=True)

%load_ext lab_black
%load_ext watermark

# Gastric cancer example

Adapted from code for [unit 8: gastric.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/gastric.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/gastric.txt).

## Associated lecture video: Unit 8, Lesson 6

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=t4pHpZxtC0U&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=87" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement
Stablein et al. (1981) provide data on 90 patients affected by locally advanced, nonresectable gastric carcinoma. The patients are randomized to two treatments: chemotherapy alone (coded as 0) and chemotherapy plus radiation (coded as 1). Survival time is reported in days. Recorded times are censored if the patient stopped participating in the study before it finished.

Stablein, D. M., Carter, W. H., Novak, J. W. (1981). Analysis of survival data with nonproportional hazard functions. Control. Clin. Trials,  2 , 2, 149--159.

### Data
Columns are, from left to right:
- type: Treatment type, chemotherapy (0) or chemotherapy + radiation (1)
- censored: If censored, meaning the patient survived the observation period, the time in days appears here rather than in the times column. 0 if not censored.
- times: Recorded days without cancer recurrence. NaN if censored.

### Model changes
I didn't implement S, f, or h from the original model. They should be simple enough, but I really just wanted to get another example of censoring up before HW6. I will add those later.

PyMC really did not like the noninformative exponential prior on v (α in this model). To avoid the divide by zero errors, I just kept increasing lambda until the model ran all the way through. This is not ideal, but I haven't had time to look into it further. The results actually came out fairly close to the BUGS results.


## Method 1: ```pm.Censored```

The way PyMC censoring works is described in some detail in [this notebook](https://docs.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-truncated-censored-regression.html#censored-regression-model) by [Dr. Benjamin T. Vincent](https://github.com/drbenvincent). This is accomplished in the source code [here](https://github.com/aesara-devs/aeppl/blob/751979802f1aef5478fdbf7cc1839df07df60825/aeppl/truncation.py#L79) if you want to take a look. For right-censoring, try this: ```pm.Censored("name", dist, lower=None, upper=censored, observed=y)```. The censored values can be an array of the same shape as the y values. 

If the y value equals the right-censored value, [```pm.Censored```](https://docs.pymc.io/en/latest/api/distributions/generated/pymc.Censored.html#pymc.Censored) returns the complement to the CDF evaluated at the censored value. If the y value is greater than the censored value, it returns ```-np.inf```. Otherwise, the distribution you passed to the ```dist``` parameter works as normal. What I've been doing is setting the values in the censored array to ```np.inf``` if the corresponding y value is not censored, and equal to the y value if it should be censored.

```{note}
I've noticed that this method is unstable with some distributions. Try using the imputed censoring model (below) if this one isn't working.
```

In [14]:
data = np.loadtxt("../data/gastric.txt")
data.shape

(90, 3)

In [15]:
x = data[:, 0].copy()
censored = data[:, 1].copy()
y = data[:, 2].copy()
# for pymc, right-censored values must be greater than or equal to than the "upper" value
y[np.isnan(y)] = censored[np.isnan(y)]
censored[censored == 0] = np.inf

In [16]:
y

array([  17.,   42.,   44.,   48.,   60.,   72.,   74.,   95.,  103.,
        108.,  122.,  144.,  167.,  170.,  183.,  185.,  193.,  195.,
        197.,  208.,  234.,  235.,  254.,  307.,  315.,  401.,  445.,
        464.,  484.,  528.,  542.,  567.,  577.,  580.,  795.,  855.,
        882.,  892., 1031., 1033., 1306., 1335., 1366., 1452., 1472.,
          1.,   63.,  105.,  129.,  182.,  216.,  250.,  262.,  301.,
        301.,  342.,  354.,  356.,  358.,  380.,  381.,  383.,  383.,
        388.,  394.,  408.,  460.,  489.,  499.,  524.,  529.,  535.,
        562.,  675.,  676.,  748.,  748.,  778.,  786.,  797.,  945.,
        955.,  968., 1180., 1245., 1271., 1277., 1397., 1512., 1519.])

In [17]:
censored

array([  inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
        882.,  892., 1031., 1033., 1306., 1335.,   inf, 1452., 1472.,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,  381.,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,  529.,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,  945.,
         inf,   inf, 1180.,   inf,   inf, 1277., 1397., 1512., 1519.])

```{warning}
PyMC and BUGS do not specify the Weibull distribution in the same way!

α = v

β = λ ** (-1 / α)

```

In [19]:
log2 = np.log(2)

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.01)
    beta1 = pm.Normal("beta1", 0, tau=0.01)
    α = pm.Exponential("α", 3)

    λ = exp(beta0 + beta1 * x)
    β = λ ** (-1 / α)

    obs_latent = pm.Weibull.dist(alpha=α, beta=β)
    likelihood = pm.Censored(
        "likelihood",
        obs_latent,
        lower=None,
        upper=censored,
        observed=y,
    )

    median0 = pm.Deterministic("median0", (log2 * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (log2 * exp(-beta0 - beta1)) ** (1 / α))

    trace = pm.sample(
        10000,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag_grad",
        step=[pm.NUTS(target_accept=0.9)],
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, α]


Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 64 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 11 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [20]:
az.summary(trace, hdi_prob=0.9)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-6.599,0.657,-7.632,-5.488,0.008,0.005,7694.0,8409.0,1.0
beta1,0.260,0.237,-0.130,0.651,0.002,0.001,20398.0,16749.0,1.0
α,0.999,0.096,0.843,1.159,0.001,0.001,7864.0,9257.0,1.0
median0,519.012,90.800,369.481,661.673,0.625,0.443,21251.0,24643.0,1.0
median1,399.620,71.604,284.827,513.780,0.483,0.341,21455.0,22387.0,1.0


## Method 2: Imputed Censoring

This method is from [this notebook](https://docs.pymc.io/projects/examples/en/latest/survival_analysis/censored_data.html#censored-data-model1) by [Luis Mario Domenzain](https://github.com/domenzain), [George Ho](https://github.com/eigenfoo), and [Dr. Ben Vincent](https://github.com/drbenvincent). This method imputes the values using what is almost another likelihood (not sure if it actually meets the definition of one, so I'm calling the variable ```impute_censored```) in the model, with the right-censored values as lower bounds. Since the two likelihoods share the same priors, this ends up working nearly as well as the previous method.

In [9]:
data = np.loadtxt("../data/gastric.txt")
x = data[:, 0].copy()
censored_vals = data[:, 1].copy()
y = data[:, 2].copy()

# we need to separate the observed values and the censored values
observed_mask = censored_vals == 0

censored = censored_vals[~observed_mask]
y_uncensored = y[observed_mask]
x_censored = x[~observed_mask]
x_uncensored = x[observed_mask]

In [10]:
log2 = np.log(2)

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.0001)
    beta1 = pm.Normal("beta1", 0, tau=0.0001)
    α = pm.Exponential("α", 3)

    λ_censored = exp(beta0 + beta1 * x_censored)
    β_censored = λ_censored ** (-1 / α)

    λ_uncensored = exp(beta0 + beta1 * x_uncensored)
    β_uncensored = λ_uncensored ** (-1 / α)

    impute_censored = pm.Bound(
        "impute_censored",
        pm.Weibull.dist(alpha=α, beta=β_censored),
        lower=censored,
        shape=censored.shape[0],
    )

    likelihood = pm.Weibull(
        "likelihood",
        alpha=α,
        beta=β_uncensored,
        observed=y_uncensored,
        shape=y_uncensored.shape[0],
    )

    median0 = pm.Deterministic("median0", (log2 * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (log2 * exp(-beta0 - beta1)) ** (1 / α))

    trace = pm.sample(
        10000, tune=2000, cores=4, init="auto", step=[pm.NUTS(target_accept=0.95)]
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, α, impute_censored]


Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 95 seconds.
There were 5 divergences after tuning. Increase `target_accept` or reparameterize.
There were 699 divergences after tuning. Increase `target_accept` or reparameterize.
There were 407 divergences after tuning. Increase `target_accept` or reparameterize.
There were 1056 divergences after tuning. Increase `target_accept` or reparameterize.


In [11]:
az.summary(trace, hdi_prob=0.9)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-6.623,0.654,-7.669,-5.522,0.007,0.005,10152.0,14532.0,1.0
beta1,0.262,0.235,-0.119,0.655,0.002,0.001,19268.0,19355.0,1.0
α,1.002,0.096,0.847,1.160,0.001,0.001,10093.0,14778.0,1.0
impute_censored[0],1472.780,636.127,882.070,2255.653,4.054,2.867,14953.0,12009.0,1.0
impute_censored[1],1487.949,641.204,892.057,2255.269,3.751,2.652,18049.0,13539.0,1.0
impute_censored[2],1622.217,635.952,1031.003,2382.012,3.792,2.681,17036.0,12023.0,1.0
impute_censored[3],1627.546,633.158,1033.007,2415.428,4.164,2.944,14628.0,10610.0,1.0
impute_censored[4],1898.058,647.580,1306.017,2686.712,4.502,3.183,12044.0,8742.0,1.0
impute_censored[5],1930.314,642.605,1335.015,2715.673,3.719,2.630,18234.0,11669.0,1.0
impute_censored[6],2038.692,638.144,1452.022,2813.702,3.710,2.623,17958.0,12279.0,1.0


In [12]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

pymc : 4.0.0
numpy: 1.22.4
arviz: 0.12.1

